In [1]:
!git clone https://github.com/mandarjoshi90/coref.git
%cd coref

Cloning into 'coref'...
remote: Enumerating objects: 734, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 734 (delta 2), reused 0 (delta 0), pack-reused 728
Receiving objects: 100% (734/734), 4.17 MiB | 8.78 MiB/s, done.
Resolving deltas: 100% (441/441), done.
/content/coref


In [3]:
train_split = 0.75
dev_split = 0.15
test_split = 0.1
seg_count = 10

In [2]:
# Installing dependencies (takes about 5 minutes)
import os
! sed 's/MarkupSafe==1.0/MarkupSafe==1.1.1/; s/scikit-learn==0.19.1/scikit-learn==0.21/; s/scipy==1.0.0/scipy==1.6.2/' < requirements.txt > tmp
! mv tmp requirements.txt

! sed 's/.D.GLIBCXX.USE.CXX11.ABI.0//' < setup_all.sh  > tmp
! mv tmp setup_all.sh 
! chmod u+x setup_all.sh 

model_name = "bert_base"
os.environ['data_dir'] = "."
os.environ['CHOSEN_MODEL'] = model_name
%tensorflow_version 2.x
! pip uninstall -y tensorflow
! pip install -r requirements.txt --log install-log.txt -q
! ./setup_all.sh

Uninstalling tensorflow-2.5.0:
  Successfully uninstalled tensorflow-2.5.0
     |████████████████████████████████| 102kB 2.2MB/s 
     |████████████████████████████████| 1.2MB 3.2MB/s 
     |████████████████████████████████| 163kB 7.0MB/s 
     |████████████████████████████████| 6.6MB 7.3MB/s 
     |████████████████████████████████| 552kB 16.5MB/s 
     |████████████████████████████████| 61kB 5.3MB/s 
     |████████████████████████████████| 2.2MB 34.0MB/s 
     |████████████████████████████████| 266kB 27.3MB/s 
     |████████████████████████████████| 890kB 36.3MB/s 
     |████████████████████████████████| 133kB 39.7MB/s 
     |████████████████████████████████| 153kB 39.0MB/s 
     |████████████████████████████████| 51kB 5.5MB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 92kB 10.4MB/s 
     |████████████████████████████████| 20.3MB 5.8MB/s 
     |████████████████████████████████| 2.1MB 39.2MB/s 
     |████████████████████████████████| 

In [4]:
# Create new train_dev_test and line splits
# Default is 751510 and 11lines
# Skip this cell if you already have sufficient train,dev,test conll files
!unzip craft_conll.zip
import glob
import re

def remove_discontinuous(filename):
    with open(filename) as f:
        text = f.readlines()
    text = [t.split("\t") for t in text if t != "\n"]
    doc_num = text[1][0]
    line_len = len(text[1])
    temp_line = []
    edited_text = []
    for t in text:
        temp_coref = t[-1].split("|")
        edited_coref = []
        for e in temp_coref:
            if e.islower():
                edited_coref.append("-")
            else:
                edited_coref.append(e)
        if len(t) == line_len:
            if len(edited_coref) > 1:
                edited_coref = [e for e in edited_coref if e != "-"]
            if edited_coref:
                edited_text.append('\t'.join(t[:10]).replace(doc_num, "nw/{}".format(doc_num)) + "\t*\t*\t*\t" + '|'.join(edited_coref) + "\n") 
            else:
                edited_text.append('\t'.join(t[:10]).replace(doc_num, "nw/{}".format(doc_num)) + "\t*\t*\t*\t" + '-' + "\n") 
        else:
            edited_text.append(t[0].replace(doc_num, "nw/{}".format(doc_num)) + "\n")
    edited_text = [l.replace("\n\n", "\n") for l in edited_text]
    edited_text.append("#end document\n")
    return edited_text 

files = glob.glob("conll_coref/*.conll")
train_percentage = int(len(files)*train_split)
dev_percentage = int(len(files)*dev_split)
test_percentage = int(len(files)*test_split)
train_text = [remove_discontinuous(t) for t in files[:train_percentage]]
dev_text = [remove_discontinuous(t) for t in files[train_percentage:train_percentage+dev_percentage]]
test_text = [remove_discontinuous(t) for t in files[train_percentage+dev_percentage:]]

def add_begin_end(temp_text,
                  seg_count=10):
    insert_begin = temp_text[0]
    insert_end = temp_text[-1]
    temp_len = len(temp_text)
    increase = 0
    count = 0
    part_num = 1
    segment_counter = 0
    for i in range(0,temp_len):
        if count < temp_len-2:
            count = i + increase
            first_split = temp_text[count+1].split("\t")
            second_split = temp_text[count].split("\t")
            if len(first_split) == 14 and len(second_split) == 14:
                if int(first_split[2]) < int(second_split[2]):
                    segment_counter += 1
                    if segment_counter == seg_count:
                        temp_text.insert(count+1, insert_end)
                        if part_num < 10:
                            temp_text.insert(count+2, insert_begin.replace("part 000", "part 00{}".format(part_num)))
                        elif part_num > 9 and part_num < 100:
                            temp_text.insert(count+2, insert_begin.replace("part 000", "part 0{}".format(part_num)))
                        else:
                            temp_text.insert(count+2, insert_begin.replace("part 000", "part {}".format(part_num)))
                        part_num += 1
                        increase += 2
                        segment_counter = 0
    return temp_text

def join_lists(temp_list):
    full_list = []
    for element in temp_list:
        for line in element:
            full_list.append(line)
    return full_list

def remove_long_links(temp_text):
    docs_list = []
    temp_doc = []
    single_coref = "\(\d+\)"
    start_coref = "\(\d+$"
    end_coref = "^d+\)"
    for line in temp_text:
        if line.startswith("#begin"):
            temp_doc.append(line)
        elif line.startswith("#end"):
            temp_doc.append(line)
            docs_list.append(temp_doc)
            temp_doc = []
        else:
            temp_doc.append(line)
    
    
    edited_docs = []
    for doc in docs_list:
        edited_doc = []
        all_doc_corefs = []
        for line in doc:
            line_split = line.split("\t")
            if len(line_split) == 14:
                if line_split[-1] != "-\n":
                    line_coref = line_split[-1].strip("\n")
                    if "|" in line_coref:
                        for e in line_coref.split("|"):
                            all_doc_corefs.append(e)
                    else:
                        all_doc_corefs.append(line_coref)
    
        for line in doc:
            if len(line.split("\t")) == 14:
                temp_coref = line.split("\t")[-1].strip("\n").split("|")
                edited_coref = []
                for tc in temp_coref:
                    if re.search(single_coref, tc):
                        edited_coref.append(tc)
                    elif re.search(start_coref, tc):
                        find_end = "{})".format(tc.split("(")[1])
                        if find_end in all_doc_corefs:
                            edited_coref.append(tc)
                        else:
                            edited_coref.append("-")
                    elif re.search(end_coref, tc):
                        find_start = "({}".format(tc.split(")"[0]))
                        if find_start in all_doc_corefs:
                            edited_coref.append(tc)
                        else:
                            edited_coref.append("-") 
                    else:
                        edited_coref.append(tc)                           
                if len(edited_coref) > 1 and "-" not in edited_coref:
                    edited_doc.append('\t'.join(line.split("\t")[:-1]) + "\t" + "|".join(edited_coref) + "\n")
                elif len(edited_coref) == 1:
                    edited_doc.append('\t'.join(line.split("\t")[:-1]) + "\t" + edited_coref[0] + "\n")
            else:
                edited_doc.append(line)
        edited_docs.append(edited_doc)  
    return edited_docs


train_final = []
for t in train_text:
    temp_list = t
    temp_list = add_begin_end(t,seg_count=seg_count)
    temp_list = remove_long_links(t)
    for line in temp_list:
        train_final.append(line)

dev_final = []
for t in dev_text:
    temp_list = t
    temp_list = add_begin_end(t, seg_count=seg_count)
    temp_list = remove_long_links(t)
    for line in temp_list:
        dev_final.append(line)

test_final = []
for t in test_text:
    temp_list = t
    temp_list = add_begin_end(t,seg_count=seg_count)
    temp_list = remove_long_links(t)
    for line in temp_list:
        test_final.append(line)
with open("train.english.v4_gold_conll", "w") as f:
    f.writelines(join_lists(train_text))
with open("dev.english.v4_gold_conll", "w") as f:
    f.writelines(join_lists(dev_text))
with open("test.english.v4_gold_conll", "w") as f:
    f.writelines(join_lists(test_text))

Archive:  craft_conll.zip
   creating: conll_coref/
  inflating: conll_coref/11319941.conll  
  inflating: conll_coref/11532192.conll  
  inflating: conll_coref/11597317.conll  
  inflating: conll_coref/11604102.conll  
  inflating: conll_coref/11897010.conll  
  inflating: conll_coref/12079497.conll  
  inflating: conll_coref/12546709.conll  
  inflating: conll_coref/12585968.conll  
  inflating: conll_coref/12925238.conll  
  inflating: conll_coref/14609438.conll  
  inflating: conll_coref/14611657.conll  
  inflating: conll_coref/14624252.conll  
  inflating: conll_coref/14675480.conll  
  inflating: conll_coref/14691534.conll  
  inflating: conll_coref/14723793.conll  
  inflating: conll_coref/14737183.conll  
  inflating: conll_coref/15005800.conll  
  inflating: conll_coref/15018652.conll  
  inflating: conll_coref/15040800.conll  
  inflating: conll_coref/15061865.conll  
  inflating: conll_coref/15070402.conll  
  inflating: conll_coref/15207008.conll  
  inflating: conll_coref

In [5]:
# Replace code for train, dev, test
def word_replace(text,
                 search_word,
                 replace_word):
    edited_text = []
    replace_word = "\t{}\t".format(replace_word)
    for line in text:        
        temp_split = line.split("\t")
        if len(temp_split) == 14:
            if temp_split[3].lower() == search_word and temp_split[-1] == "-\n":
                temp_search = "\t{}\t".format(temp_split[3])
                edited_text.append(line.replace(temp_search, replace_word))
            else:
                edited_text.append(line)
        else:
            edited_text.append(line)
    return edited_text

In [6]:
edited_split_docs = []
!mkdir edited_tdt
files = ["train.english.v4_gold_conll", 
         "dev.english.v4_gold_conll",
         "test.english.v4_gold_conll"]
for split_file in files:
    with open(split_file) as f:
        text = f.readlines()
    # A list of the pleonastic pronouns to be replaced
    words = ["it"]
    # Non-word to replace the pleonastic pronouns
    replacement_word = "XxXxX"
    temp = text
    for word in words:
        temp = word_replace(temp, word, replacement_word)
    edited_split_docs.append(temp)
    with open("edited_tdt/{}".format(split_file), "w") as f:
        f.writelines(temp) 

!zip -r edited_it_751510_11lines_wparts edited_tdt

  adding: edited_tdt/ (stored 0%)
  adding: edited_tdt/train.english.v4_gold_conll (deflated 89%)
  adding: edited_tdt/dev.english.v4_gold_conll (deflated 89%)
  adding: edited_tdt/test.english.v4_gold_conll (deflated 89%)


In [9]:
# Code to convert CoNLL to jsonlines format 
# This code as been adapted from minimize.py contained in https://github.com/mandarjoshi90/coref.git
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import re
import os
import sys
import json
import tempfile
import subprocess
import collections

import util
import conll
from bert import tokenization

class DocumentState(object):
  def __init__(self, key):
    self.doc_key = key
    self.sentence_end = []
    self.token_end = []
    self.tokens = []
    self.subtokens = []
    self.info = []
    self.segments = []
    self.subtoken_map = []
    self.segment_subtoken_map = []
    self.sentence_map = []
    self.pronouns = []
    self.clusters = collections.defaultdict(list)
    self.coref_stacks = collections.defaultdict(list)
    self.speakers = []
    self.segment_info = []

  def finalize(self):
    # finalized: segments, segment_subtoken_map
    # populate speakers from info
    subtoken_idx = 0
    for segment in self.segment_info:
      speakers = []
      for i, tok_info in enumerate(segment):
        if tok_info is None and (i == 0 or i == len(segment) - 1):
          speakers.append('[SPL]')
        elif tok_info is None:
          speakers.append(speakers[-1])
        else:
          speakers.append(tok_info[9])
          if tok_info[4] == 'PRP':
            self.pronouns.append(subtoken_idx)
        subtoken_idx += 1
      self.speakers += [speakers]
    # populate sentence map

    # populate clusters
    first_subtoken_index = -1
    for seg_idx, segment in enumerate(self.segment_info):
      speakers = []
      for i, tok_info in enumerate(segment):
        first_subtoken_index += 1
        coref = tok_info[-2] if tok_info is not None else '-'
        if coref != "-":
          last_subtoken_index = first_subtoken_index + tok_info[-1] - 1
          for part in coref.split("|"):
            if part[0] == "(":
              if part[-1] == ")":
                cluster_id = int(part[1:-1])
                self.clusters[cluster_id].append((first_subtoken_index, last_subtoken_index))
              else:
                cluster_id = int(part[1:])
                self.coref_stacks[cluster_id].append(first_subtoken_index)
            else:
              cluster_id = int(part[:-1])
              try:
                start = self.coref_stacks[cluster_id].pop()
                self.clusters[cluster_id].append((start, last_subtoken_index))
              except:
                #print("Skipping {}".format(seg_idx))  
                pass
    # merge clusters
    merged_clusters = []
    for c1 in self.clusters.values():
      existing = None
      for m in c1:
        for c2 in merged_clusters:
          if m in c2:
            existing = c2
            break
        if existing is not None:
          break
      if existing is not None:
        print("Merging clusters (shouldn't happen very often.)")
        existing.update(c1)
      else:
        merged_clusters.append(set(c1))
    merged_clusters = [list(c) for c in merged_clusters]
    all_mentions = util.flatten(merged_clusters)
    sentence_map =  get_sentence_map(self.segments, self.sentence_end)
    subtoken_map = util.flatten(self.segment_subtoken_map)
    assert len(all_mentions) == len(set(all_mentions))
    num_words =  len(util.flatten(self.segments))
    assert num_words == len(util.flatten(self.speakers))
    assert num_words == len(subtoken_map), (num_words, len(subtoken_map))
    assert num_words == len(sentence_map), (num_words, len(sentence_map))
    return {
      "doc_key": self.doc_key,
      "sentences": self.segments,
      "speakers": self.speakers,
      "constituents": [],
      "ner": [],
      "clusters": merged_clusters,
      'sentence_map':sentence_map,
      "subtoken_map": subtoken_map,
      'pronouns': self.pronouns
    }


def normalize_word(word, language):
  if language == "arabic":
    word = word[:word.find("#")]
  if word == "/." or word == "/?":
    return word[1:]
  else:
    return word

# first try to satisfy constraints1, and if not possible, constraints2.
def split_into_segments(document_state, max_segment_len, constraints1, constraints2):
  current = 0
  previous_token = 0
  while current < len(document_state.subtokens):
    end = min(current + max_segment_len - 1 - 2, len(document_state.subtokens) - 1)
    while end >= current and not constraints1[end]:
      end -= 1
    if end < current:
        end = min(current + max_segment_len - 1 - 2, len(document_state.subtokens) - 1)
        while end >= current and not constraints2[end]:
            end -= 1
        if end < current:
            raise Exception("Can't find valid segment")
    document_state.segments.append(['[CLS]'] + document_state.subtokens[current:end + 1] + ['[SEP]'])
    subtoken_map = document_state.subtoken_map[current : end + 1]
    document_state.segment_subtoken_map.append([previous_token] + subtoken_map + [subtoken_map[-1]])
    info = document_state.info[current : end + 1]
    document_state.segment_info.append([None] + info + [None])
    current = end + 1
    previous_token = subtoken_map[-1]

def get_sentence_map(segments, sentence_end):
  current = 0
  sent_map = []
  sent_end_idx = 0
  assert len(sentence_end) == sum([len(s) -2 for s in segments])
  for segment in segments:
    sent_map.append(current)
    for i in range(len(segment) - 2):
      sent_map.append(current)
      current += int(sentence_end[sent_end_idx])
      sent_end_idx += 1
    sent_map.append(current)
  return sent_map

def get_document(document_lines, tokenizer, language, segment_len):
  document_state = DocumentState(document_lines[0])
  word_idx = -1
  for line in document_lines[1]:
    row = line.split()
    sentence_end = len(row) == 0
    if not sentence_end:
      assert len(row) >= 12
      word_idx += 1
      word = normalize_word(row[3], language)
      subtokens = tokenizer.tokenize(word)
      document_state.tokens.append(word)
      document_state.token_end += ([False] * (len(subtokens) - 1)) + [True]
      for sidx, subtoken in enumerate(subtokens):
        document_state.subtokens.append(subtoken)
        info = None if sidx != 0 else (row + [len(subtokens)])
        document_state.info.append(info)
        document_state.sentence_end.append(False)
        document_state.subtoken_map.append(word_idx)
    else:
      document_state.sentence_end[-1] = True
  # split_into_segments(document_state, segment_len, document_state.token_end)
  # split_into_segments(document_state, segment_len, document_state.sentence_end)
  constraints1 = document_state.sentence_end if language != 'arabic' else document_state.token_end
  split_into_segments(document_state, segment_len, constraints1, document_state.token_end)
  stats["max_sent_len_{}".format(language)] = max(max([len(s) for s in document_state.segments]), stats["max_sent_len_{}".format(language)])
  document = document_state.finalize()
  return document

def skip(doc_key):
  # if doc_key in ['nw/xinhua/00/chtb_0078_0', 'wb/eng/00/eng_0004_1']: #, 'nw/xinhua/01/chtb_0194_0', 'nw/xinhua/01/chtb_0157_0']:
    # return True
  return False

def minimize_partition(name, language, extension, labels, stats, tokenizer, seg_len, input_dir, output_dir):
  input_path = "{}/{}.{}.{}".format(input_dir, name, language, extension)
  output_path = "{}/{}.{}.{}.jsonlines".format(output_dir, name, language, seg_len)
  count = 0
  print("Minimizing {}".format(input_path))
  documents = []
  with open(input_path, "r") as input_file:
    for line in input_file.readlines():
      begin_document_match = re.match(conll.BEGIN_DOCUMENT_REGEX, line)
      if begin_document_match:
        doc_key = conll.get_doc_key(begin_document_match.group(1), begin_document_match.group(2))
        documents.append((doc_key, []))
      elif line.startswith("#end document"):
        continue
      else:
        documents[-1][1].append(line)
  with open(output_path, "w") as output_file:
    for document_lines in documents:
      if skip(document_lines[0]):
        continue
      document = get_document(document_lines, tokenizer, language, seg_len)
      output_file.write(json.dumps(document))
      output_file.write("\n")
      count += 1
  print("Wrote {} documents to {}".format(count, output_path))

def minimize_language(language, labels, stats, vocab_file, seg_len, input_dir, output_dir, do_lower_case):
  # do_lower_case = True if 'chinese' in vocab_file else False
  tokenizer = tokenization.FullTokenizer(
                vocab_file=vocab_file, do_lower_case=do_lower_case)
  minimize_partition("dev", language, "v4_gold_conll", labels, stats, tokenizer, seg_len, input_dir, output_dir)
  minimize_partition("train", language, "v4_gold_conll", labels, stats, tokenizer, seg_len, input_dir, output_dir)
  minimize_partition("test", language, "v4_gold_conll", labels, stats, tokenizer, seg_len, input_dir, output_dir)


vocab_file = "cased_config_vocab/vocab.txt"
input_dir = "."
output_dir = "."
do_lower_case = 'true'
print(do_lower_case)
labels = collections.defaultdict(set)
stats = collections.defaultdict(int)
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)
for seg_len in [128, 256, 384, 512]:
    minimize_language("english", labels, stats, vocab_file, seg_len, input_dir, output_dir, do_lower_case)
    # minimize_language("chinese", labels, stats, vocab_file, seg_len)
    # minimize_language("es", labels, stats, vocab_file, seg_len)
    # minimize_language("arabic", labels, stats, vocab_file, seg_len)
for k, v in labels.items():
    print("{} = [{}]".format(k, ", ".join("\"{}\"".format(label) for label in v)))
for k, v in stats.items():
    print("{} = {}".format(k, v))


true
Minimizing ./dev.english.v4_gold_conll
Wrote 435 documents to ./dev.english.128.jsonlines
Minimizing ./train.english.v4_gold_conll
Wrote 2238 documents to ./train.english.128.jsonlines
Minimizing ./test.english.v4_gold_conll
Wrote 333 documents to ./test.english.128.jsonlines
Minimizing ./dev.english.v4_gold_conll
Wrote 435 documents to ./dev.english.256.jsonlines
Minimizing ./train.english.v4_gold_conll
Wrote 2238 documents to ./train.english.256.jsonlines
Minimizing ./test.english.v4_gold_conll
Wrote 333 documents to ./test.english.256.jsonlines
Minimizing ./dev.english.v4_gold_conll
Wrote 435 documents to ./dev.english.384.jsonlines
Minimizing ./train.english.v4_gold_conll
Wrote 2238 documents to ./train.english.384.jsonlines
Minimizing ./test.english.v4_gold_conll
Wrote 333 documents to ./test.english.384.jsonlines
Minimizing ./dev.english.v4_gold_conll
Wrote 435 documents to ./dev.english.512.jsonlines
Minimizing ./train.english.v4_gold_conll
Wrote 2238 documents to ./train.e